#### 一. 同步异步与阻塞非阻塞的区分
1. 同步异步: 从2个任务的执行顺序上区分  
    * 同步: 指一个任务, 只有当另一个任务返回后才能继续执行本任务
    * 异步: 指一个任务只是发起一个通知告诉另一个任务可以执行了, 然后就继续执行自身的任务
    
 同步可以保证2个任务的执行顺序, 而异步不能
 
 
2. 阻塞和非阻塞: 从1个线程等待另一个线程返回时的状态区分
    * 阻塞: 一个线程如果在等待另一个线程返回时, 自身处于挂起状态, 就是阻塞的  
    * 非阻塞: 反之则是非阻塞
    
    
3. 同步阻塞和同步非阻塞  
 同步表示必须保证任务顺序, 阻塞表示线程等待另一个线程时是否会被挂起, 因此:
    * 同步非阻塞: 一个线程必须等待一个方法调用完毕才能继续执行接下来的任务, 且在等待过程中还可以执行其他方法, 则是同步非阻塞的
    * 同步阻塞:  一个线程必须等待一个方法调用完毕才能继续执行接下来的任务, 但是等待过程中自己处于挂起状态, 则是同步阻塞的

#### 二. 几个概念
1. 用户空间和内核空间  
现代操作系统使用虚拟存储器([软硬件接口/5-存储器层级结构/5.7-虚拟存储器]()), 虚拟内存将主存划分为内核空间和用户空间


2. Linux IO模型  
   用户进程在读取/写入外存上的数据时, 由于使用了虚拟存储器技术, 首先会把外存的数据映射到内存上(将虚拟地址中的页映射到物理地址的页上), 此过程发生2次拷贝:  
    * 为了保证系统安全, 操作系统会首先把虚拟地址中的内容拷贝到内核空间 (外存->内存)
    * 再把该内核空间的数据拷贝到用户空间(进行虚拟地址映射的物理地址)上 (内存->内存, recvFrom命令)  
    
 对socket传来的数据而言, 数据先从socket拷贝到内核空间, 然后再从内核空间拷贝到用户空间


#### 三. Linux的几种IO模型
#### 同步IO模型
1. **同步阻塞IO**   
   上面已经说明, socket的数据要传给用户进程需要两步: 
    * socket复制到内核空间
    * 内核空间复制到用户空间
    
 同步阻塞IO的意思是说, 这两个步骤的数据复制时, 用户进程都是处于阻塞状态, 不能做任何处理  
 ```
 所以，blocking IO的特点就是在数据复制的两个阶段都被block了。
 ```
<img src="img/20150405_VKYH.png" width="60%">
 
 
2. **同步非阻塞IO**  
   当socket被设置成非阻塞的后, 用户进程要获取socket的数据时, 第一步socket复制数据到内核空间过程中, 用户进程不会阻塞, 只会被操作系统返回一个错误码, 表示数据还未准备好. 于是用户进程要轮训询问操作系统, 内核空间的数据是否已经复制完毕, 等复制完毕后, 用户进程进入阻塞状态, 等待数据从内核空间复制到用户空间   
   ```
   所以，nonblocking IO的特点是用户进程需要不断的主动询问kernel数据好了没有。
   ```
<img src="img/20152818_DXcj.png" width="70%">
   
   
3. **IO多路复用(AIO)**  
   &nbsp;&nbsp;&nbsp;&nbsp;上面说到, 同步非阻塞模式需要用户进程不断询问操作系统, 内核空间是否已经把数据从socket复制完毕. 如果这个轮训是操作系统自动完成, 且一次可以同时轮训多个socket的数据是否复制完毕就好了. 于是Linux出现了epool(pool,select的升级版)  
   &nbsp;&nbsp;&nbsp;&nbsp;每个socket连接, 在Linux系统中会被表示成一个文件描述符(fd, 文件描述符也用来表示一个进程),IO多路复用操作过程如下:  
    1. 用户进程调用select系统调用后进入阻塞状态
    2. 操作系统开始轮训所有socket fd, 当任何一个socket fd的数据可读/可写后(不是拷贝完/写完到内核空间), select调用就会返回  
    3. 用户进程在select调用返回后变成就绪状态, 调用recvFrom调用, 将数据从内核空间拷贝到用户空间
    
 &nbsp;&nbsp;&nbsp;&nbsp;IO多路复用主要解决服务器同时存在大量连接的情况, 是单线程处理多连接的模型; 与传统的多线程处理多连接, 一方面减小了多线程的开销, 另一面避免场链接带来的长时间阻塞问题  
    ```
    第一个阶段(socket->内核空间)可以阻塞又可以不阻塞。(根据poll,select, epoll的不同而不同)
    第二个阶段(内核空间->用户空间)是阻塞的。
    ```
<img src="img/20164149_LD8E.png" width="70%">


4. **信号驱动IO**    
信号驱动式I/O：首先我们允许Socket进行信号驱动IO,并安装一个信号处理函数，进程继续运行并不阻塞。当数据准备好时，进程会收到一个SIGIO信号，可以在信号处理函数中调用I/O操作函数处理数据。过程如下图所示：
<img src="img/21091434_DsZb.png" width="70%">


#### 异步IO模型    
 5. 异步非阻塞IO(AIO)    
     异步阻塞IO是说, 用户发起aio_read操作之后, 函数立刻返回, 用户进程开始做其它工作, socket数据复制到内核空间, 和内核空间复制到用户空间这两个步骤都是操作系统完成的. 两步完成后, 操作系统会给用户进程发送一个signal或执行一个基于线程的回调函数来完成这次 IO 处理过程，告诉它read操作完成了.
<img src="img/20175459_gtgw.png" width="70%">

#### 几种IO之间的区别
<img src="img/21095604_vhHX.png" width="70%">